In [1]:
import base64
from pathlib import Path

import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa
import numpy as np
import pandas as pd
import panel as pn
import textwrap


# ---------------------------------------------------------
# 1. Extensions
# ---------------------------------------------------------
pn.extension("tabulator", sizing_mode="stretch_width")
hv.extension("bokeh")

# ---------------------------------------------------------
# 2. Paths and assets
# ---------------------------------------------------------
DATA_PATH = "results/final_E2SFCA_results.geojson"
RECOMMENDED_SITES_PATH = "results/recommended_new_sites.geojson"
# Background images
IMG_CDC_AVOID = "figures/background/cdc_avoid_care.png"
IMG_KEY_EVIDENCE = "figures/background/key_evidence_table.png"
IMG_CLOSED_SIGN = "figures/background/closed_sign.png"
IMG_LONG_LINE = "figures/background/long_queue.png"
IMG_PIPELINE = "figures/overview/data_pipeline.png"
IMG_DRIVE_TIME = "figures/overview/drive_time_nearest.png"
IMG_ACCESS_PARADOX = "figures/overview/access_paradox.png" 

IMG_EDGE = "results/01_missteps_edge_effects/01_error_edge_effect_demonstration_fixed.png"
IMG_SVI_MAP = "results/02_intermediate_process/02_process_SVI_distribution.png"
IMG_POP_DENSITY = "results/02_intermediate_process/02_process_pop_density.png"
IMG_SCATTER = "results/cluster/scatter_high_contrast.png"
IMG_BOXPLOT = "results/cluster/boxplot_high_contrast.png"
IMG_LEGEND = "final_deliverables/publication_maps_v5/03_typologies_legend_forced.png"
IMG_EQUITY_MAP = "results/03_final_deliverables/03_final_equity_gap_map_optimized.png"
IMG_SUPPLY_TABLE = "figures/data_tables/supply_table.png"
IMG_DEMAND_TABLE = "figures/data_tables/demand_table.png"
IMG_SPATIAL_TABLE = "figures/data_tables/spatial_network_table.png"


def encode_image(path: str, width: int = 1200):
    """
    Embed an image as base64 and make it large and centered on the page.
    """
    p = Path(path)
    if not p.exists():
        return pn.pane.Markdown(f"**Image not found:** `{path}`")

    with p.open("rb") as f:
        data = base64.b64encode(f.read()).decode("utf-8")

    html = (
        f'<img src="data:image/png;base64,{data}" '
        f'style="max-width:{width}px; width:100%; height:auto; '
        f'display:block; margin-left:auto; margin-right:auto; border-radius:6px;">'
    )
    return pn.pane.HTML(html)



# ---------------------------------------------------------
# 3. Load and prepare data
# ---------------------------------------------------------
print("Loading tract-level accessibility dataset...")
gdf = gpd.read_file(DATA_PATH).to_crs("EPSG:4326")

# Common map extent (keep all city tracts in view, zoomed in)
xmin, ymin, xmax, ymax = gdf.total_bounds
x_pad = (xmax - xmin) * 0.05
y_pad = (ymax - ymin) * 0.05
X_RANGE = (xmin - x_pad, xmax + x_pad)
Y_RANGE = (ymin - y_pad, ymax + y_pad)

# SVI
if "RPL_THEMES" in gdf.columns:
    gdf["SVI"] = gdf["RPL_THEMES"].replace(-999, 0.5).fillna(0.5)
elif "SVI" not in gdf.columns:
    gdf["SVI"] = 0.5

# Accessibility for Medicaid and all providers
if "access_medicaid" in gdf.columns:
    gdf["Access_Score"] = gdf["access_medicaid"]
elif "Access_Score" not in gdf.columns:
    gdf["Access_Score"] = 0.0

if "access_all" not in gdf.columns:
    gdf["access_all"] = gdf["Access_Score"]

# Equity gap: percent loss of E2SFCA when restricted to Medicaid providers
gdf["Equity_Gap"] = (
    (gdf["access_all"] - gdf["Access_Score"])
    / gdf["access_all"].replace(0, 0.0001)
    * 100
).clip(0, 100).fillna(0)

# Cluster names
if "Cluster_ID" in gdf.columns:
    cluster_labels = {0: "Critical Desert", 1: "Privileged", 2: "Supported", 3: "Moderate"}
    gdf["Cluster_Name"] = gdf["Cluster_ID"].map(cluster_labels).fillna("Unknown")
elif "Cluster_Name" not in gdf.columns:
    gdf["Cluster_Name"] = "N/A"

# Weighted impact (priority index)
if "Weighted_Impact" not in gdf.columns:
    eg_max = float(gdf["Equity_Gap"].max())
    if eg_max == 0 or np.isnan(eg_max):
        eg_max = 1.0
    eg_norm = gdf["Equity_Gap"] / eg_max

    svi_max = float(gdf["SVI"].max())
    if svi_max == 0 or np.isnan(svi_max):
        svi_max = 1.0
    svi_norm = gdf["SVI"] / svi_max

    gdf["Weighted_Impact"] = eg_norm * svi_norm

# IDs and names
if "GEOID" not in gdf.columns:
    gdf["GEOID"] = gdf.index.astype(str)
if "NAME" in gdf.columns and "Tract_Name" not in gdf.columns:
    gdf["Tract_Name"] = gdf["NAME"]
elif "Tract_Name" not in gdf.columns:
    gdf["Tract_Name"] = gdf["GEOID"]

# Recommended sites (three candidate locations)
recommended_sites = None
sites_path = Path(RECOMMENDED_SITES_PATH)
if sites_path.exists():
    recommended_sites = gpd.read_file(sites_path).to_crs(gdf.crs)
    if "site_id" not in recommended_sites.columns:
        recommended_sites["site_id"] = recommended_sites.index.astype(str)
else:
    recommended_sites = None

# Flat DataFrame for non-spatial widgets
df_flat = pd.DataFrame(gdf.drop(columns="geometry"))

# ---------------------------------------------------------
# 4. Interactive components
# ---------------------------------------------------------
metric_options = {
    "Equity Gap (% access lost)": "Equity_Gap",
    "Weighted Impact (Priority index)": "Weighted_Impact",
    "Social Vulnerability Index (SVI)": "SVI",
    "Accessibility Score (Medicaid E2SFCA)": "Access_Score",
}
metric_select = pn.widgets.Select(
    name="Select metric",
    options=metric_options,
    value="Equity_Gap",
)


@pn.depends(metric_select)
def metric_map(metric_col: str):
    """
    Choropleth map of the selected metric.
    """
    col = metric_col
    df = gdf.dropna(subset=[col])
    vmin, vmax = float(df[col].min()), float(df[col].max())
    title = f"{col} [Scale: {vmin:.1f} – {vmax:.1f}]"

    hover_cols = [
        c
        for c in [
            "GEOID",
            "Tract_Name",
            "Cluster_Name",
            "SVI",
            "Access_Score",
            "Equity_Gap",
            "Weighted_Impact",
        ]
        if c in df.columns
    ]

    return df.hvplot.polygons(
        geo=True,
        tiles="CartoLight",
        c=col,
        cmap="Viridis",
        alpha=0.9,
        line_width=0.1,
        colorbar=True,
        frame_width=900,
        frame_height=620,
        title=title,
        hover_cols=hover_cols,
        xlim=X_RANGE,
        ylim=Y_RANGE,
    )


def scatter_with_regression():
    """
    Scatter of SVI vs accessibility, with a fitted regression line.
    """
    df = df_flat.dropna(subset=["SVI", "Access_Score"]).copy()

    if df.empty:
        return pn.pane.Markdown("No data available for scatter plot.")

    scatter = df.hvplot.scatter(
        x="SVI",
        y="Access_Score",
        by="Cluster_Name",
        cmap="Category10",
        size=50,
        alpha=0.7,
        legend="top_right",
        hover_cols=["GEOID", "Tract_Name", "Cluster_Name"],
        frame_width=820,
        frame_height=520,
        title="Accessibility vs Social Vulnerability (with regression line)",
        xlabel="Social Vulnerability Index (SVI)",
        ylabel="Accessibility score (Medicaid E2SFCA)",
    )

    x = df["SVI"].values
    y = df["Access_Score"].values
    if len(df) < 2:
        return scatter

    coeffs = np.polyfit(x, y, 1)
    x_line = np.linspace(x.min(), x.max(), 100)
    y_line = coeffs[0] * x_line + coeffs[1]
    reg_line = hv.Curve((x_line, y_line)).opts(
        color="black",
        line_width=2,
        line_dash="dashed",
        show_legend=False,
    )

    return scatter * reg_line


scatter_pane = scatter_with_regression()


def siting_map():
    """
    Priority surface plus recommended new sites.
    """
    base_col = "Weighted_Impact" if "Weighted_Impact" in gdf.columns else "Equity_Gap"
    hover_cols = [
        c
        for c in [
            "GEOID",
            "Tract_Name",
            "SVI",
            "Access_Score",
            "Equity_Gap",
            "Weighted_Impact",
        ]
        if c in gdf.columns
    ]

    base = gdf.hvplot.polygons(
        geo=True,
        tiles="CartoLight",
        c=base_col,
        cmap="Purples",
        alpha=0.8,
        line_width=0.05,
        colorbar=True,
        frame_width=900,
        frame_height=620,
        title="Priority surface with recommended new sites",
        hover_cols=hover_cols,
        xlim=X_RANGE,
        ylim=Y_RANGE,
    )

    if recommended_sites is None or recommended_sites.empty:
        return base

    sites_hover = [c for c in recommended_sites.columns if c != "geometry"]

    points = recommended_sites.hvplot.points(
        geo=True,
        marker="star",
        size=180,
        color="red",
        alpha=0.95,
        hover_cols=sites_hover,
    )

    return base * points


siting_map_pane = siting_map()

# Table of indicators
table_cols = [
    c
    for c in [
        "GEOID",
        "Tract_Name",
        "Cluster_Name",
        "SVI",
        "Access_Score",
        "Equity_Gap",
        "Weighted_Impact",
    ]
    if c in df_flat.columns
]
table_pane = pn.widgets.Tabulator(
    df_flat[table_cols],
    pagination="remote",
    page_size=15,
    height=420,
    sizing_mode="stretch_width",
)

# ---------------------------------------------------------
# 5. Page layouts (tabs)
# ---------------------------------------------------------
base_style = {"padding": "25px", "background": "#ffffff"}

# Background tab
tab_overview = pn.Column(
    pn.pane.Markdown("# Project overview"),
    pn.pane.Markdown(
        """
This final project for **MUSA 5500 Geospatial Data Science in Python** uses
Philadelphia as a case study to examine how spatial accessibility to acute
care changes once we move beyond simple distance and incorporate capacity,
insurance acceptance, and social vulnerability.
"""
    ),

    # --------------------------------------------------
    # Research questions
    # --------------------------------------------------
    pn.pane.Markdown("### Research questions"),
    pn.pane.Markdown(
        """
<span style="color:#b30000;"><strong>RQ1 (Diagnosis):</strong> How does spatial accessibility to acute healthcare facilities vary across Philadelphia when measured using a realistic road network and an E2SFCA travel time model under free flow conditions? (Addressed by Levels 1 and 2.)</span>  

<span style="color:#b30000;"><strong>RQ2 (Equity):</strong> Is this accessibility equitably distributed, or does it systematically disadvantage socially vulnerable groups? (Addressed by Level 4.)</span>  

<span style="color:#b30000;"><strong>RQ3 (Intervention):</strong> Where should new facilities be located to maximize impact for the most vulnerable? (Addressed by Level 3 logic.)</span>
"""
    ),

    # --------------------------------------------------
    # Hypotheses
    # --------------------------------------------------
    pn.pane.Markdown("### Hypotheses"),
    pn.pane.Markdown(
        """
**H1 [Beyond distance].** Even when realistic road network travel times are used, variations in spatial accessibility will not be primarily driven by physical distance to hospitals, but by capacity constraints and spatial mismatches between supply and demand.  

**H2 [The Medicaid gap].** Vulnerable populations will face a significant accessibility penalty when limited to Medicaid accepting facilities, confirming structural inequity.  

**H3 [Targeted optimization].** Siting new facilities based on a multi criteria "Gap Index" that combines population with SVI will be more effective than traditional density based planning.
"""
    ),

    pn.layout.Divider(),

    # --------------------------------------------------
    # Five analytical levels, with larger heading
    # --------------------------------------------------
    pn.pane.Markdown(
        "<h3 style='font-size:22px;'>Five analytical levels</h3>"
    ),
    pn.pane.Markdown(
        """
**Level 1, naive distance and p median**  
*Method:* Euclidean distance or shortest path travel time and a simple p median allocation.  
*Role:* Baseline that illustrates the "distance myth", physical proximity alone does not guarantee real access because capacity is ignored.

**Level 2, interaction via E2SFCA**  
*Method:* Enhanced Two Step Floating Catchment Area with Gaussian distance decay.  
*Role:* Core engine that quantifies accessibility as a capacity adjusted ratio on the road network, integrating supply and demand within realistic catchments.

**Level 3, optimization and gap logic**  
*Method:* Light demand gap index and simple maximal coverage reasoning, selecting a small set of candidate sites.  
*Role:* Scenario tool that tests how three to five new facilities under a budget constraint could shift access for high priority tracts.

**Level 4, equity integration**  
*Method:* Social Vulnerability Index and Medicaid only filtering applied to Level 2 scores.  
*Role:* Turns accessibility into a distributive justice question, asking who loses how much access and where vulnerable groups are systematically penalized.

**Level 5, emerging AI and robustness**  
*Method:* Predictive machine learning and simple stochastic surge models.  
*Role:* Frames a longer term research agenda that can predict hidden demand and stress test the system under future shocks, beyond the scope of a single masters thesis.
"""
    ),

    pn.layout.Divider(),

    # --------------------------------------------------
    # Position of the current work
    # --------------------------------------------------
    pn.pane.Markdown(
        "<h3 style='font-size:22px;'>Position of the current work</h3>"
    ),
    pn.pane.Markdown(
        """
**Core of the thesis**

- **Level 2** [E2SFCA] provides the minimum scientifically credible measure of spatial accessibility.  
- **Level 4** [equity integration with SVI and Medicaid] translates accessibility into explicit distributive justice questions, connecting the model to planning practice.

**Baseline and teaching tools**

- **Level 1** [naive distance and p median] serves as a baseline that demonstrates the "distance myth".  
  It is valuable for teaching and communication, but it is not sufficient for serious policy guidance.

**Optional extensions**

- **Level 3** [a light MCLP style siting of three to five facilities] is implemented as scenario analysis.  
  It helps explore intervention options, but it is not required to answer the core diagnostic and equity questions.  
- **Level 5** [predictive or robust optimization] is outlined as future work, for example using machine learning to infer hidden demand or simple stochastic models of surge events.

**Message for the committee**

Levels **2** and **4** form the smallest, necessary combination that fully answers **RQ1** and **RQ2** for this thesis.  
Level **1** anchors the analysis in intuitive baselines, while Levels **3** and **5** define a realistic longer term research agenda that can be pursued beyond this project.
"""
    ),

    name="📊 Overview",
    styles=base_style,
)


# ---------------------------------------------------------
# 4A. Reproducible data acquisition snippets (collapsible)
# ---------------------------------------------------------

code_census_api = pn.pane.Markdown(
    """```python
import requests
import pandas as pd

CENSUS_API_KEY = "YOUR_KEY_HERE"

BASE_URL = "https://api.census.gov/data/2021/acs/acs5"
VARS = ["B01003_001E",  # total population
        "C27012_004E"]  # Medicaid or public insurance

params = {
    "get": ",".join(["NAME"] + VARS),
    "for": "tract:*",
    "in": "state:42 county:101",  # Philadelphia County
    "key": CENSUS_API_KEY,
}

r = requests.get(BASE_URL, params=params)
r.raise_for_status()
data = r.json()

acs_df = pd.DataFrame(data[1:], columns=data[0])
acs_df["GEOID"] = acs_df["state"] + acs_df["county"] + acs_df["tract"]
acs_df = acs_df.drop(columns=["state", "county", "tract"])
```""",
    sizing_mode="stretch_width",
)

code_svi_download = pn.pane.Markdown(
    """```python
import requests
from pathlib import Path
import geopandas as gpd

SVI_URL = "https://svi.cdc.gov/Documents/Data/2020_SVI_Data/SVI2020_tract.gdb.zip"
out_zip = Path("data/raw/svi_2020.zip")
out_dir = Path("data/raw/svi_2020")

out_zip.parent.mkdir(parents=True, exist_ok=True)

if not out_zip.exists():
    r = requests.get(SVI_URL)
    r.raise_for_status()
    out_zip.write_bytes(r.content)

# Extract manually or with zipfile, then read tract layer
svi_gdf = gpd.read_file(out_dir, layer="SVI2020_US")  # layer name depends on CDC release
# Keep only Philadelphia County (42101)
svi_philly = svi_gdf[svi_gdf["FIPS"].str.startswith("42101")]
```""",
    sizing_mode="stretch_width",
)

code_osmnx_network = pn.pane.Markdown(
    """```python
import osmnx as ox
import numpy as np
import pandas as pd

# 1. Build drivable network inside a buffered bounding box
place_name = "Philadelphia, Pennsylvania, USA"
G = ox.graph_from_place(place_name, network_type="drive", buffer_dist=15000)

# 2. Get tract centroids as demand points
tracts = gdf.to_crs(G.graph["crs"])
tracts["centroid"] = tracts.geometry.centroid

# 3. Compute travel time in minutes to all hospitals within 30 minutes
# (Hospitals and health centers pre-merged in a GeoDataFrame called facilities)
facilities = facilities.to_crs(G.graph["crs"])
facilities["node"] = ox.nearest_nodes(G, facilities.geometry.x, facilities.geometry.y)
tracts["node"] = ox.nearest_nodes(G, tracts.centroid.x, tracts.centroid.y)

# Example: nearest provider travel time (Level 1)
G_speed = ox.add_edge_speeds(G)
G_tt = ox.add_edge_travel_times(G_speed)

def travel_time_minutes(orig, dest):
    try:
        route = ox.shortest_path(G_tt, orig, dest, weight="travel_time")
        edges = ox.utils_graph.get_route_edge_attributes(G_tt, route, "travel_time")
        tt = sum(edges)
        return tt / 60.0
    except Exception:
        return np.nan

# In practice the project uses an OD matrix, this is a simplified illustration.
```""",
    sizing_mode="stretch_width",
)

data_code_accordion = pn.Accordion(
    ("1.3.a ACS and Medicaid via Census API", code_census_api),
    ("1.3.b CDC SVI geodatabase download", code_svi_download),
    ("1.3.c OSMnx network and travel-time matrix", code_osmnx_network),
)
data_code_accordion.active = []  # all collapsed by default

# ---------------------------------------------------------
# 5. Data & methods tab
# ---------------------------------------------------------
tab_methods = pn.Column(
    pn.pane.Markdown("# Data and methods"),

    # --------------------------------------------------
    # 1. Data overview
    # --------------------------------------------------
    pn.pane.Markdown("## 1. Data overview"),

    pn.pane.Markdown("### 1.1 Supply-side data"),
    encode_image(IMG_SUPPLY_TABLE, width=980),
    pn.pane.Markdown(
        """
The supply table lists all datasets that describe **where care is provided**:

- **Hospitals and acute-care facilities.**  
  Core supply points, merged from OpenDataPhilly, state health agencies, CMS
  and OpenStreetMap, then de-duplicated by name and address.  

- **City and community health centers.**  
  Safety-net providers scraped from the Philadelphia Department of Public
  Health website to fill gaps in official open data.  

- **Buffer-area hospitals.**  
  Facilities in surrounding counties that fall within the network catchment
  of Philadelphia tracts, these are essential for correcting **edge effects**.  

- **Hospital bed capacity.**  
  Used as a proxy for acute-care capacity when computing supply to demand
  ratios in the E2SFCA model.
"""
    ),

    pn.pane.Markdown("### 1.2 Demand-side data"),
    encode_image(IMG_DEMAND_TABLE, width=980),
    pn.pane.Markdown(
        """
Demand-side datasets describe **who needs care** and **how vulnerable they are**:

- **Total population by tract (ACS 2021).**  
  Pulled via the U.S. Census API and used as baseline demand in all
  accessibility calculations.  

- **Social Vulnerability Index (SVI, CDC ATSDR).**  
  Provides a tract-level composite index in the range 0-1 capturing
  socioeconomic status, minority status, housing and transportation
  vulnerability.  

- **Medicaid share.**  
  Derived from ACS insurance tables and used to approximate the share of
  residents relying on public insurance.  

- **WorldPop population grid.**  
  A high-resolution raster downloaded manually from WorldPop.org and used to
  cross-check ACS population counts through zonal statistics.
"""
    ),

    pn.pane.Markdown("### 1.3 Basic spatial and network data"),
    encode_image(IMG_SPATIAL_TABLE, width=980),
    pn.pane.Markdown(
        """
Spatial and network layers provide the **geographic backbone** for the model:

- **Census tract boundaries, including neighbors.**  
  TIGER/Line shapefiles, downloaded and processed programmatically, with an
  extended region that covers neighboring counties for edge effect correction.  

- **Road network (driving graph).**  
  A drivable street network extracted from OpenStreetMap via OSMnx, including
  speed estimates and travel times.  

- **Travel-time OD matrix (minutes).**  
  Origin destination travel times in minutes between tract centroids and
  facilities, computed on the OSMnx graph.  

- **HUD Qualified Census Tracts (QCT).**  
  Used as an additional vulnerability layer to identify historically
  disadvantaged tracts.  

- **City and county boundary.**  
  Used only for cartographic masking, to avoid artificially truncating
  catchments at the study area border.
"""
    ),

    pn.layout.Divider(),

    # --------------------------------------------------
    # 1.4 Data acquisition methods
    # --------------------------------------------------
    pn.pane.Markdown("## 1.4 Data acquisition methods"),
    pn.pane.Markdown(
        """
**(a) APIs and web services**

- **U.S. Census Bureau API.**  
  `requests` is used to download ACS 2021 5-year estimates for population and
  Medicaid related insurance variables at the tract level.  

- **ArcGIS REST API (HUD QCT).**  
  Qualified Census Tracts are queried directly as GeoJSON, filtered to the
  extended Philadelphia region, and saved to disk for reproducibility.  

- **Nominatim geocoding API.**  
  Geocodes scraped health center addresses into coordinates using `geopy`,
  which are then projected and snapped to the street network.

**(b) Web scraping**

- **BeautifulSoup scraper for city health centers.**  
  Because some safety net facilities are only listed on an HTML page,
  a small `BeautifulSoup` script parses the table, standardizes addresses,
  and writes them into a clean CSV before geocoding.

**(c) Specialized Python libraries**

- **OSMnx.**  
  Downloads the drivable road network, adds edge speeds and travel times, and
  provides nearest node mapping from centroids and facilities to the graph.  

- **GeoPandas and Rasterio.**  
  Used to read shapefiles, project geometries, and perform zonal statistics on
  the WorldPop raster to validate ACS populations.

**(d) Static archives and manual downloads**

Some datasets are intentionally **manually sourced**, then documented and read
from disk:

- **WorldPop raster (`usa_pop_2020...tif`).**  
  Downloaded from WorldPop.org, stored under `data/raw/`, and read only inside
  a helper function (`process_worldpop_raster`). There is no download script
  in this repository.  

- **CDC SVI CSV (`philly_svi_2020.csv`).**  
  Downloaded from CDC and ATSDR, then read via
  `pd.read_csv(DATA_RAW / "philly_svi_2020.csv")`.  
  This category is labeled as *Manual download* in the data tables.
"""
    ),

    pn.layout.Divider(),

    # --------------------------------------------------
    # 1.5 Reproducible acquisition snippets
    # --------------------------------------------------
    pn.pane.Markdown("## 1.5 Reproducible acquisition snippets"),
    pn.pane.Markdown(
        """
The accordion below contains simplified, non executable code excerpts that
show how key datasets are fetched. The full versions live in the project
notebooks and scripts.
"""
    ),
    data_code_accordion,

    pn.layout.Divider(),

    # --------------------------------------------------
    # 2. Methods
    # --------------------------------------------------
    pn.pane.Markdown("## 2. Methods"),

    # 2.1 Preprocessing and integration
    pn.pane.Markdown("### 2.1 Preprocessing and integration"),
    pn.pane.Markdown(
        """
1. **Cleaning and standardising IDs.**  
   All tract level datasets (ACS, SVI, QCT) are keyed by `GEOID`, while
   facility datasets are keyed by a stable `facility_id`. Duplicates and
   obvious geocoding errors are removed.  

2. **Merging supply layers.**  
   Hospitals, buffer area hospitals, and community health centers are
   concatenated into a single facility layer with attributes for type,
   Medicaid acceptance, and bed capacity.  

3. **Validating population counts.**  
   WorldPop raster counts are aggregated to tracts using zonal statistics
   and compared to ACS totals. Large discrepancies are flagged, but ACS
   remains the authoritative source for the final model.  

4. **Constructing vulnerability and insurance indicators.**  
   SVI (`RPL_THEMES`) is cleaned and re scaled into `SVI` in the range 0-1.  
   Medicaid share is derived from ACS insurance tables and joined to the
   tract layer, yielding a flat `df_flat` table used for clustering
   and scatter plots.
"""
    ),

    # 2.2 Network analysis and E2SFCA
    pn.pane.Markdown("### 2.2 Network analysis and accessibility modelling"),

    pn.pane.Markdown("#### 2.2.1 Analytical pipeline"),
    encode_image(IMG_PIPELINE, width=980),
    pn.pane.Markdown(
        """
The conceptual pipeline summarises the analytical flow:

1. **Data collection** via APIs, scraping and manual downloads.  
2. **Preprocessing and integration** of supply, demand and vulnerability data.  
3. **Network analysis and E2SFCA** on an OSMnx driving graph to compute
   accessibility metrics.  
4. **Clustering and dashboarding** to turn raw indicators into typologies
   and interactive exploration tools.
"""
    ),

    pn.pane.Markdown("#### 2.2.2 Baseline drive time and the edge effect problem"),
    encode_image(IMG_DRIVE_TIME, width=980),
    pn.pane.Markdown(
        """
This map shows **drive time to the nearest provider** computed only from
hospitals inside the Philadelphia city boundary. Several features signal a
methodological problem:

- Many tracts, especially in the northern and eastern parts of the city,
  appear to be 4-9 minutes away from the nearest provider.  
- The pattern is strongly shaped by the city boundary, suggesting that
  facilities just across the line are being ignored.

In reality, residents near the edge routinely cross city and county borders
for care. Treating the administrative boundary as a hard wall therefore
overstates travel times and creates **artificial deserts**.
"""
    ),

    encode_image(IMG_EDGE, width=980),
    pn.pane.Markdown(
        """
To correct this, the model is re run with a **buffer region of facilities**:

- A rectangular bounding box is defined around Philadelphia using the tract
  layer extent plus a generous margin.  
- All hospitals and health centers inside this box are fetched and merged
  into the supply layer.  
- The origin destination matrix and nearest provider times are recomputed on
  the buffered network.

The revised travel time surface no longer shows deserts that are purely an
artefact of the study area extent, and becomes a consistent input to the
E2SFCA model.
"""
    ),

    pn.pane.Markdown("#### 2.2.3 E2SFCA accessibility with Gaussian decay"),
    pn.pane.Markdown(
        """
The **Enhanced Two Step Floating Catchment Area (E2SFCA)** model is implemented
as the core accessibility metric:

1. **Step 1, facility catchment ratios.**  
   For each facility *j*, identify all tracts *i* reachable within 30 minutes,
   and compute a supply to demand ratio  

   \\( R_j = S_j / \\sum_i D_i w(d_{ij}) \\)  

   where *S_j* is bed capacity, *D_i* is tract demand (population or Medicaid
   population), and \\( w(d_{ij}) \\) is a **Gaussian distance decay** that
   down weights longer trips.

2. **Step 2, tract accessibility scores.**  
   For each tract *i*, sum the ratios of all reachable facilities  

   \\( A_i = \\sum_j R_j w(d_{ij}) \\)  

   to obtain an accessibility score that reflects both **competition for beds**
   and **travel effort**.

The model is run twice:

- **All facilities (`access_all`).**  
- **Medicaid accepting facilities only (`Access_Score`).**

The **Equity_Gap** is defined as the percentage loss in E2SFCA when access is
restricted to Medicaid facilities, and a **Weighted_Impact** index combines
Equity_Gap with SVI to highlight tracts where the loss falls on highly
vulnerable populations.
"""
    ),

    # 2.3 Machine-learning clustering and typologies
    pn.pane.Markdown("### 2.3 Machine learning clustering and neighborhood typologies"),

    encode_image(IMG_LEGEND, width=980),
    pn.pane.Markdown(
        """
This stage introduces the **machine learning component** of the project.

Using **KMeans clustering from scikit-learn**, the model performs unsupervised
clustering on tract level indicators:

- **Input features.**  
  SVI, Medicaid constrained E2SFCA accessibility, and in some variants the
  Equity_Gap. All features are standardised before clustering.  

- **Algorithm.**  
  `KMeans(n_clusters=4, random_state=...)` is applied to identify groups of
  tracts with similar combinations of vulnerability and access. The choice
  of four clusters balances interpretability and within cluster variance.

- **Resulting neighborhood types (as used in the legend above).**  
  - **High SVI / Low Access – Critical deserts.**  
    Vulnerable residents with weak effective access, primary targets for new
    facilities in the siting scenarios.  
  - **High SVI / High Access – Supported but vulnerable.**  
    Vulnerable groups located near large medical campuses, barriers are more
    institutional and financial than spatial.  
  - **Low SVI / High Access – Privileged.**  
    Well served, low vulnerability areas that currently benefit the most from
    existing infrastructure.  
  - **Low SVI / Low Access – Low access but less vulnerable.**  
    Tracts with weaker access but lower overall social vulnerability.

These machine learning typologies operationalise **RQ2 (Equity)** by showing
*who* loses access, not just *where* accessibility is low. They connect
directly to the **Priority Index** and the recommended new sites in the
Results tab, where high priority corridors of critical deserts guide the
placement of additional capacity.
"""
    ),

    name="🛠 Data & Methods",
    styles=base_style,
)


# ---------------------------------------------------------
# 5. Results tab
# ---------------------------------------------------------
tab_results = pn.Column(
    pn.pane.Markdown("# 3. Results"),

    pn.pane.Markdown(
        """
This section answers **RQ1–RQ3** and evaluates **H1–H3** using the final
E2SFCA accessibility surfaces, the Medicaid equity gap, and the
priority-based siting scenarios.
"""
    ),

    pn.layout.Divider(),

    # 3.1 Medicaid equity gap
    pn.pane.Markdown("### 3.1 The Medicaid equity gap (RQ2, H2)"),
    encode_image(IMG_EQUITY_MAP, width=980),
    pn.pane.Markdown(
        """
The map above shows **percentage accessibility loss** when the network is
restricted to **Medicaid-accepting facilities only**. Darker reds represent
tracts where **over a quarter of E2SFCA access is lost** because nearby
hospitals or clinics do not accept Medicaid.

Key patterns:

- Large parts of **Southwest, Lower North, and River-ward Philadelphia**
  show very high loss, even though they are not far from major hospitals.  
- Many **high-SVI tracts** fall in the upper loss classes, confirming the
  hypothesized **“Medicaid equity gap”** in **H2**.  
- By contrast, some tracts around Center City and near major campuses
  retain most of their access even under Medicaid constraints.

Together, these results support **H2** and provide direct evidence that
insurance barriers significantly reshape effective accessibility, not just
physical distance.
"""
    ),

    pn.layout.Divider(),

    # 3.2 Clustered accessibility and vulnerability
    pn.pane.Markdown("### 3.2 Neighborhood typologies (RQ2, H1)"),
    encode_image(IMG_SCATTER, width=980),
    pn.pane.Markdown(
        """
The scatter plot summarizes **cluster results** from K-Means on
E2SFCA accessibility and SVI:

- <span style="color:#2ca02c;"><strong>Privileged (green)</strong></span>:  
  **Low SVI, high access.** These tracts combine strong hospital coverage
  with relatively low social vulnerability.  
- <span style="color:#1f77b4;"><strong>Supported (blue)</strong></span>:  
  **High SVI, high access.** Vulnerable but buffered by proximity to
  large medical campuses.  
- <span style="color:#ff7f0e;"><strong>Moderate access (orange)</strong></span>:  
  **High SVI, mid-level access.** These areas are at risk of becoming
  deserts if capacity tightens.  
- <span style="color:#d62728;"><strong>Critical deserts (red)</strong></span>:  
  **High SVI, low access.** These tracts sit in the worst combination of
  social vulnerability and limited E2SFCA accessibility.

The cloud of points and weak relationship between SVI and access
reinforce **H1**: even with realistic network travel times, **capacity and
insurance acceptance** explain much of the variation, not simple distance.
"""
    ),

    pn.layout.Divider(),

    # 3.3 Priority index and site recommendations
    pn.pane.Markdown("### 3.3 Priority index and site recommendations (RQ3, H3)"),
    pn.pane.Markdown(
        """
To guide **RQ3 (Intervention)**, the project defines a **Priority Index**:

> **Priority Index = Normalized Equity Gap × Normalized SVI**

High values flag tracts that are both **highly vulnerable** and **lose
a large share of access** under Medicaid constraints. This surface is then
used as a **demand-weighting layer** for siting analysis.
"""
    ),
    siting_map_pane,
    pn.pane.Markdown(
        """
The map overlays the Priority Index surface with three **candidate stars**:

- Sites are chosen from the **upper tail of the Priority Index** distribution
  to capture clusters of critical deserts.  
- Locations are deliberately spread across **north, west, and southwest**
  Philadelphia to avoid over-serving any single corridor.  
- This supports **H3**, showing that a **multi-criteria “gap index”** focuses
  investment on systematically disadvantaged neighborhoods, rather than
  simply chasing population density or existing hospital clusters.

These recommendations should be read as **scenario results**, not fixed
facility locations, but they illustrate how Level 3 logic can translate
quantitative gaps into concrete planning options.
"""
    ),

    pn.layout.Divider(),

    # 3.4 Synthesis, future directions, conclusions, limitations
    pn.pane.Markdown("### 3.4 Future directions (by analytical level)"),
    pn.pane.Markdown(
        """
- **Core of the thesis:**  
  - **Level 2 (E2SFCA)** provides the minimum scientifically credible
    measure of spatial accessibility.  
  - **Level 4 (SVI + Medicaid)** turns accessibility into a question of
    **distributive justice**, answering **RQ1 and RQ2**.  

- **Baselines and teaching tools:**  
  - **Level 1** distance metrics and P-median baselines help debunk the
    **“distance myth”** that proximity automatically implies access.  

- **Optional extensions:**  
  - **Level 3** offers a light **MCLP-style** scenario (siting 3–5 facilities
    under a budget constraint) to answer **RQ3** more formally.  
  - **Level 5** points toward predictive and robust optimization, for example
    machine-learning models of hidden demand or stochastic surge capacity.

<span style="color:#b30000;"><strong>Message for the committee:</strong></span>  
**Level 2 + Level 4** form the smallest necessary combination that fully
answers **RQ1 and RQ2**, while **Level 3 and Level 5** define a realistic
research agenda beyond a single master’s thesis.
"""
    ),

    pn.layout.Divider(),

    pn.pane.Markdown("### 3.5 Conclusions"),
    pn.pane.Markdown(
        """
- **High-SVI neighborhoods face a double burden:** they suffer both from
  higher COVID-19 exposure risk and **lower effective access** to acute care.  
- **Policy interventions must explicitly target “medical deserts”** and
  incentivize **Medicaid acceptance** in high-priority clusters, not only
  expand capacity in already well-served areas.  
- Future equity-oriented siting should treat **accessibility, vulnerability,
  and insurance** as joint design constraints, not separate silos.
"""
    ),

    pn.layout.Divider(),

    pn.pane.Markdown("### 3.6 Limitations"),
    pn.pane.Markdown(
        """
- **Temporal bias:** travel times are computed under **free-flow conditions**,
  ignoring congestion and time-of-day variation.  
- **Single mode:** only driving is modelled; **transit and walking** are not
  yet included, which likely understates inequities for car-less households.  
- **Capacity proxy:** hospital **beds** are used as a rough capacity proxy;
  appointment slots and staffing patterns were unavailable.  
- **Insurance modelling:** Medicaid is treated as a **binary acceptance flag**,
  not a share of capacity reserved for publicly insured patients.  
- **Edge uncertainty:** cross-state referral patterns and out-of-city care
  are simplified, even though patients in border tracts may routinely cross
  county or state lines.

These caveats suggest that the numerical values should be interpreted as
**comparative indicators**, not precise forecasts, but the spatial patterns
and inequities are robust across multiple specifications.
"""
    ),

    name="📊 Results",
    styles=base_style,
)


# ---------------------------------------------------------
# 6. Dashboard tab
# ---------------------------------------------------------
tab_dashboard = pn.Column(
    pn.pane.Markdown("# 4. Interactive dashboard"),

    pn.pane.Markdown(
        """
Use this tab to **explore tract-level accessibility and equity**.

- Choose a metric in the left-hand selector to update the map.  
- Hover over tracts to see **GEOID, SVI, accessibility and gap values**.  
- Use the scatter plot and table to inspect individual neighborhoods.
"""
    ),

    pn.layout.Divider(),

    pn.Row(
        pn.Column(
            pn.pane.Markdown("### 4.1 Select metric"),
            metric_select,
            width=260,
        ),
        pn.Spacer(width=20),
        pn.Column(metric_map, sizing_mode="fixed"),
        sizing_mode="stretch_width",
    ),

    pn.pane.Markdown(
        """
### 4.2 How to read the four map metrics

- **Equity Gap (% access lost)**  
  Shows **how much E2SFCA accessibility disappears** when the network is
  restricted to **Medicaid-accepting providers**. Darker colors highlight
  neighborhoods where insurance acts as a strong barrier, directly linked to
  **RQ2** and **H2**.

- **Weighted Impact (Priority Index)**  
  Defined as **normalized Equity Gap × normalized SVI**. High values mark
  **vulnerable neighborhoods that also lose a large share of access**, and
  therefore drive the **siting recommendations** in the Results tab
  (**RQ3, H3**).

- **Social Vulnerability Index (SVI)**  
  Captures **underlying social risk**, including poverty, housing crowding
  and lack of vehicle access. Comparing this map with Equity Gap reveals
  where structural vulnerability is **not** compensated by better access.

- **Accessibility Score (Medicaid E2SFCA)**  
  Measures the **effective supply of Medicaid-accepting capacity per person**
  within realistic drive-time catchments. Tracts with **high SVI but low
  scores** fall into the “critical desert” category.
"""
    ),

    pn.layout.Divider(),

    pn.pane.Markdown("### 4.3 Accessibility versus SVI (interactive)"),
    pn.Row(
        pn.Spacer(width=40),
        scatter_pane,
        sizing_mode="stretch_width",
    ),
    pn.pane.Markdown(
        """
**Interpretation**

- Points in the **upper-right** indicate tracts that are both vulnerable and
  relatively well served (often near major hospitals).  
- Points in the **lower-right** combine high SVI with low accessibility and
  correspond to the **critical desert** and high-priority clusters.  
- The near-flat regression line suggests that **greater social vulnerability
  does not systematically buy better access**, reinforcing the need for
  **equity-oriented siting** rather than purely demand-driven planning.
"""
    ),

    pn.layout.Divider(),

    pn.pane.Markdown("### 4.4 Underlying tract-level indicators"),
    table_pane,
    pn.pane.Markdown(
        """
The table lists the core indicators used throughout the project:

- **SVI** and **Equity Gap** columns can be sorted to quickly find the
  highest-risk tracts.  
- **Weighted_Impact** ranks neighborhoods by **combined vulnerability and
  access loss**, mirroring the Priority Index used in the siting map.  

Users can export these values or join them back to other GIS layers for
further analysis.
"""
    ),

    name="📈 Dashboard",
    styles=base_style,
)


# ---------------------------------------------------------
# 7. References tab
# ---------------------------------------------------------
tab_references = pn.Column(
    pn.pane.Markdown("# 5. References and documentation"),

    pn.pane.Markdown(
        """
The full thesis engages with **over 200 academic and policy sources**.
Below are the **core methodological references** that anchor the models and
interpretations shown in this dashboard.
"""
    ),

    pn.pane.Markdown("### 5.1 Key methodological references"),
    pn.pane.Markdown(
        """
- **CDC. (2020).** Delay or Avoidance of Medical Care Because of COVID-19–Related Concerns. *MMWR Morbidity and Mortality Weekly Report*.  
- **Luo, W., & Qi, Y. (2009).** An enhanced two-step floating catchment area (E2SFCA) method. *Health & Place*.  
- **Wang, F. (2012).** Measurement, optimization, and impact of health care accessibility: A methodological review. *Annals of the AAG*.  
- **Planey, A. M., et al. (2022).** Locating equity: Spatial access and health equity analysis of the Medicaid-accepting safety-net. *Social Science & Medicine*.  
- **Roy, A., & Kar, B. (2022).** A multicriteria decision analysis framework to measure equitable healthcare access during COVID-19.  
- **Faraji-Najarkolaie, F., et al. (2022).** Expanding vulnerability indices for pandemic effects. *Journal of Transport & Health*.  

*Etc.* – Additional articles are cited in the written paper but omitted here
for brevity.
"""
    ),

    pn.layout.Divider(),

    pn.pane.Markdown("### 5.2 Data sources"),
    pn.pane.Markdown(
        """
- **U.S. Census Bureau, ACS 5-year estimates (2021)** – population and
  Medicaid-related insurance indicators.  
- **CDC / ATSDR Social Vulnerability Index (SVI)** – tract-level vulnerability
  scores and themes.  
- **OpenDataPhilly & State health departments** – hospital and acute-care
  facility locations and attributes.  
- **City of Philadelphia & health department websites** – community health
  centers and safety-net providers, including scraped locations.  
- **HUD Qualified Census Tracts (QCT)** – identification of low-income and
  high-poverty tracts.  
- **OpenStreetMap via OSMnx** – drivable street network for the regional
  catchment area.  
- **WorldPop raster grids** – high-resolution population surfaces used for
  cross-validation of tract-level counts.
"""
    ),

    pn.layout.Divider(),

    pn.pane.Markdown("### 5.3 Code and reproducibility"),
    pn.pane.Markdown(
        """
All scripts, notebooks, and intermediate outputs are stored in the project’s
GitHub repository. The repository is organized by **data/raw**, **data/processed**,
**results**, and **docs** folders to support full reproducibility and future
extensions (for example, transit networks or dynamic capacity modelling).
"""
    ),

    name="📖 References",
    styles=base_style,
)


# ---------------------------------------------------------
# 8. About tab
# ---------------------------------------------------------
tab_about = pn.Column(
    pn.pane.Markdown("# 6. About this project"),

    pn.pane.Markdown(
        """
**Author:** Haolong Chen  
**Program:** Master of City Planning, Smart Cities concentration  
**Institution:** University of Pennsylvania  

This dashboard is the final project for **MUSA 5500 – Geospatial Data Science
in Python** and also forms the **empirical core** of a broader thesis on
healthcare accessibility and equity in Philadelphia.
"""
    ),

    pn.pane.Markdown("### 6.1 Purpose"),
    pn.pane.Markdown(
        """
The project demonstrates how **network-based accessibility models** and
**equity metrics** can be combined to:

- Diagnose where acute-care access is structurally limited.  
- Quantify the **Medicaid equity gap**.  
- Propose **priority locations** for new or expanded facilities.

It is intended as a **teaching and exploration tool** for planners, public
health researchers, and policy makers.
"""
    ),

    pn.pane.Markdown("### 6.2 Technical stack"),
    pn.pane.Markdown(
        """
- **Python libraries:** GeoPandas, Pandas, NumPy, OSMnx, NetworkX, Rasterio,
  scikit-learn, hvPlot, Holoviews, and Panel.  
- **Visualization:** Bokeh tiles and Carto basemaps for web-native maps.  
- **Deployment:** Static HTML site generated via `panel.save()` and served
  through **GitHub Pages**.
"""
    ),

    pn.pane.Markdown("### 6.3 How to reuse or extend"),
    pn.pane.Markdown(
        """
- Clone the GitHub repository and recreate the `geo310` (or equivalent)
  conda environment.  
- Replace the input hospital and population layers with those from another
  city to replicate the workflow elsewhere.  
- Extend the code with **transit and walking networks**, **time-of-day
  travel times**, or more detailed **insurance and capacity data**.
"""
    ),

    pn.pane.Markdown("### 6.4 Disclaimer"),
    pn.pane.Markdown(
        """
This dashboard is a **student research project**, not an official product of
the City of Philadelphia, the University of Pennsylvania, or any healthcare
provider. All analyses should be interpreted as **exploratory** and are
subject to data limitations described in the Results section.
"""
    ),

    pn.pane.Markdown(
        "[GitHub repository](https://github.com/chenhl0503-create/philly-healthcare-accessibility)"
    ),

    name="ℹ️ About",
    styles=base_style,
)

# ---------------------------------------------------------
# 6. Final assembly and export
# ---------------------------------------------------------
dashboard = pn.Column(
    pn.pane.Markdown(
        "# 🏥 Philadelphia Healthcare Equity Explorer",
        styles={
            "font-size": "26px",
            "background": "#f0f0f0",
            "padding": "12px",
            "width": "100%",
        },
    ),
    pn.Tabs(
        tab_overview,
        tab_methods,
        tab_results,
        tab_dashboard,
        tab_references,
        tab_about,
        dynamic=False,
    ),
)

# IMPORTANT: output Panel HTML OUTSIDE docs/, let Quarto copy it into docs/
output_dir = Path("site_assets")
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / "dashboard.html"

print(f"Generating Panel dashboard at {output_file}...")
dashboard.save(
    str(output_file),
    embed=True,
    resources="INLINE",
    title="Philadelphia Healthcare Equity Explorer",
)
print("Done. Panel dashboard exported to site_assets/dashboard.html")


Loading tract-level accessibility dataset...
Generating Panel dashboard at site_assets\dashboard.html...
                                                                                                                       

Done. Panel dashboard exported to site_assets/dashboard.html
